In [17]:
import pickle as pkl
from lda import load_docs_in_bow, process_vectors, compute_word_topic_matrix, compute_document_topic_matrix
from query_ranking import queries_string2int, get_queries, query_score
import numpy as np
from sklearn.metrics import average_precision_score

In [18]:
N_TOPIC = 100
ALPHA = 50 / N_TOPIC
BETA = 0.01

In [19]:
path_to_doc = "../data/docs_trec_covid"

vocabulary, vectors, doc_id = load_docs_in_bow(path_to_doc, n_doc=1000)

dict_document = process_vectors(vectors)

In [20]:
path_to_learned_data = f'../data/learning_data_covid_{N_TOPIC}_topics.pkl'

with open(path_to_learned_data, 'rb') as file:
    topic_assignment, Mwt, Mdt = pkl.load(file)

In [7]:
path_qurey = '../data/queries_trec_covid'

int_queries = queries_string2int(get_queries(path_qurey), vocabulary)

queries_scores = []


for i, int_query in enumerate(int_queries):
    print(f'calculate score of {i} query')
    queries_scores.append(query_score(int_query, dict_document, Mwt, Mdt, N_TOPIC, len(vocabulary), ALPHA, BETA))

calculate score of 0 query
calculate score of 1 query
calculate score of 2 query


KeyboardInterrupt: 

In [9]:
# import logging
# for n_topic in range(100, 601, 100):
#     logging.debug(f'editing {n_topic} topics file ...')
#     path_to_learned_data = f'../data/learning_data_covid_{n_topic}_topics.pkl'
#     saved_path = f'../data/learning_data_covid_{n_topic}_topics_edited.pkl'
#     
#     with open(path_to_learned_data, 'rb') as file:
#         topic_assignment = pkl.load(file)
#         
#     Mwt = compute_word_topic_matrix(topic_assignment, n_topic, len(vocabulary))
#     Mdt = compute_document_topic_matrix(topic_assignment, n_topic)
#     
#     with open(saved_path, 'wb') as file:
#         pkl.dump((topic_assignment, Mwt, Mdt), file)

In [17]:
with open('../data/queries_scores_100_topics.pkl', 'wb') as file:
    pkl.dump(queries_scores, file)

In [59]:
path_to_qrels = "../data/qrels_trec_covid"
with open(path_to_qrels, 'r') as file:
    qrels = file.read().split('\n')[:-1]

dict_qrels = {}
for i in range(0, len(qrels), 3):
    query_id = int(qrels[i])
    doc = qrels[i+1]
    relevance = int(qrels[i+2])
    if doc in doc_id:
        int_doc = doc_id.index(doc)
        if int_doc in dict_qrels:
            dict_qrels[int_doc][query_id] = relevance
        else:
            dict_qrels[int_doc] = {query_id : relevance}

In [63]:
print(len(qrels))

207954


In [3]:
with open('../data/queries_scores_100_topics.pkl', 'rb') as file:
    queries_scores_100_topics = pkl.load(file)

In [66]:
for q, ranking in enumerate(queries_scores_100_topics):
    top10 = np.argsort(ranking)[::-1][:10]
    y_truth = []
    y_pred = []
    score = 2
    for doc in top10:
        if doc in dict_qrels.keys():
            dict_relevance_doc = dict_qrels[doc]
            if q in dict_relevance_doc.keys():
                y_pred.append(score)
                y_truth.append(dict_relevance_doc[q])
        
        score -= 0.2
        

KeyboardInterrupt: 

In [65]:
print(y_truth)

[]


In [5]:
for ranking in queries_scores_100_topics:
    top_10 = np.argsort(ranking)

[[-133.9783486650796, -133.83167402777354, -134.28779208241193, -133.68325469084795, -133.9812337472137, -134.07144316702642, -133.759869640731, -133.87006884079594, -133.98519542661762, -133.98675599065652, -133.89085987382356, -133.95063157125378, -133.7578024528642, -133.6185696683682, -132.81512497609407, -134.1585690435137, -133.76667561629284, -133.86105971936868, -134.11695808525403, -134.1456977135222, -133.38344255972927, -134.08955369795154, -133.70291510342958, -133.8738094505162, -134.18325996187298, -133.7212958401256, -133.60902521323777, -133.89743360798542, -133.95881248708056, -133.77935730804924, -134.03277256832325, -133.6359123718708, -133.75439017520435, -133.75865126520748, -133.82777332622783, -133.96883892642143, -133.8316321170257, -133.95924531456254, -133.81705604473456, -133.7456719180801, -134.07926551310905, -133.25488423674173, -133.95618303943692, -133.41125200960545, -133.46270836771114, -134.18768236704793, -134.082609782295, -133.9999140848155, -133.6